In [ ]:
import time
from airo_robots.manipulators.hardware.ur_rtde import URrtde
from airo_camera_toolkit.cameras.realsense import Realsense
from airo_spatial_algebra.se3 import SE3Container
import numpy as np
import cv2
from microplate.opencv_annotation import get_manual_annotations_image, Annotation
from microplate.camera_pose import get_camera_pose
from airo_camera_toolkit.utils import ImageConverter
from microplate.multi_view_triangulation import multiview_triangulation_midpoint, get_triangulation_errors
from airo_robots.grippers.hardware.robotiq_2f85_urcap import Robotiq2F85

# reload code

%load_ext autoreload
%autoreload 2

In [ ]:
gripper = Robotiq2F85("10.42.0.162")
gripper.open().wait()

In [ ]:
camera_pose_in_tcp = get_camera_pose()
camera_pose_in_tcp = get_camera_pose()
camera = Realsense(resolution=Realsense.RESOLUTION_720,fps =6)
robot = URrtde("10.42.0.162",URrtde.UR3E_CONFIG)



collect the views to determine the positions of the keypoints on the microplates

In [ ]:

home_pose = SE3Container.from_euler_angles_and_translation([np.pi,0,0],[0.3,0.0,0.2]).homogeneous_matrix
robot.move_linear_to_tcp_pose(home_pose,0.1).wait()
gripper.open()

target_position = np.array([0.34,-0.01,0.0]) # approximate center of the microplate

# determine the camera poses for the three views
view_position_1 = np.array([0.3,0.0,0.3])
view_position_2 = np.array([0.35,-0.1,0.3])
view_position_3 = np.array([0.3,0.1,0.3])
#view_position_4 = np.array([0.25,-0.2,0.2])

view_positions = [view_position_1,view_position_2,view_position_3]
tcp_poses = []
for view in view_positions:
    view_orientation = np.eye(3)
    z_vector = target_position - view
    z_vector /= np.linalg.norm(z_vector)
    x_vector = np.cross(z_vector,np.array([0,0,1]))
    x_vector /= np.linalg.norm(x_vector)
    y_vector = np.cross(z_vector,x_vector)
    y_vector /= np.linalg.norm(y_vector)
    view_orientation[:,0] = x_vector
    view_orientation[:,1] = y_vector
    view_orientation[:,2] = z_vector
    camera_pose = SE3Container.from_rotation_matrix_and_translation(view_orientation,view).homogeneous_matrix
    tcp_pose = camera_pose @ np.linalg.inv(camera_pose_in_tcp)
    tcp_poses.append(tcp_pose)

# collect the images from the three viewpoints
images = []
camera_poses = []
for pose in tcp_poses:
    robot.move_linear_to_tcp_pose(home_pose,0.1).wait()
    robot.move_linear_to_tcp_pose(pose,0.1).wait()
    time.sleep(1)
    image = camera.get_rgb_image()
    images.append(image)
    camera_poses.append(pose @ camera_pose_in_tcp)
    # cv2.imshow("image",image)
    # cv2.waitKey(0)
    # cv2.destroyAllWindows()

robot.move_linear_to_tcp_pose(home_pose,0.1).wait()


manual annotation of the keypoints

In [ ]:


# # annotate the images
# annotation_spec = {
#     "topleft": Annotation.Keypoint,
#     "topright": Annotation.Keypoint,
#     "bottomleft": Annotation.Keypoint,

# }

# annotations = []

# for image in images:
#     image_anns = get_manual_annotations_image(image, annotation_spec)
#     image_keypoints = []
#     for kp in annotation_spec.keys():
#         keypoint = image_anns[kp]
#         keypoint = np.array(keypoint)
#         image_keypoints.append(keypoint)
#     annotations.append(image_keypoints)
# print(annotations)

use neural network to find the keypoints on the microplate


In [ ]:
from keypoint_detection.utils.heatmap import get_keypoints_from_heatmap
from keypoint_detection.utils.load_checkpoints import get_model_from_wandb_checkpoint
import torch
model = "tlips/microplate-keypoints/model-a0dlcw4n:v10"
device = "cuda"
model = get_model_from_wandb_checkpoint(model)
model.eval()
model.to(device)

In [ ]:
# get the keypoints from the image
from airo_camera_toolkit.image_transforms import Resize
transform = Resize((1280,720),256,512)
annotations = []
for image_idx,image in enumerate(images):
    print(image.shape)
    image = transform(image)
    image = torch.Tensor(image.copy()).permute(2, 0, 1).to(device)
    with torch.no_grad():
        heatmaps = model(image.unsqueeze(0)).squeeze(0)
    predicted_keypoints = [
        torch.tensor(get_keypoints_from_heatmap(heatmaps[i].cpu(), 2,max_keypoints=1)) for i in range(heatmaps.shape[0])
    ]
    print(predicted_keypoints)
    opencv_image = image.permute(1,2,0).cpu().numpy() 
    opencv_image = ImageConverter.from_numpy_format(opencv_image).image_in_opencv_format
    for i,channel in enumerate(predicted_keypoints):
        for keypoint in channel:
            color = [0,0,0]
            color[i] = 255
            color = tuple(color)
            cv2.circle(opencv_image,(int(keypoint[0]),int(keypoint[1])),5,color,-1)
    cv2.imwrite(f"detected_keypoints_{image_idx}.png",opencv_image)
    #annotations.append([transform.reverse_transform_point(channel[0]) for channel in predicted_keypoints])


Multi-view triangulation of the keypoints

In [ ]:
camera_intrinsics = camera.intrinsics_matrix()
view_extrinsics = camera_poses
topleft_image_points = [annotation[0] for annotation in annotations]
topright_image_points = [annotation[1] for annotation in annotations]
bottomleft_image_points = [annotation[2] for annotation in annotations]
topleft_midpoint = multiview_triangulation_midpoint(view_extrinsics, [camera_intrinsics]*len(view_extrinsics), topleft_image_points)
topright_midpoint = multiview_triangulation_midpoint(view_extrinsics, [camera_intrinsics]*len(view_extrinsics), topright_image_points)
bottomleft_midpoint = multiview_triangulation_midpoint(view_extrinsics, [camera_intrinsics]*len(view_extrinsics), bottomleft_image_points)

In [ ]:
print(f"{topleft_midpoint=}")
print(f"{topright_midpoint=}")
print(f"{bottomleft_midpoint=}")
print(get_triangulation_errors(view_extrinsics, [camera_intrinsics] * len(view_extrinsics), topleft_image_points, topleft_midpoint))

Define the grasp pose of the microplate in the local frame

In [ ]:
micropplate_x_axis = topright_midpoint - topleft_midpoint
micropplate_x_axis /= np.linalg.norm(micropplate_x_axis)
micropplate_y_axis = topleft_midpoint - bottomleft_midpoint
micropplate_y_axis /= np.linalg.norm(micropplate_y_axis)
micropplate_z_axis = np.cross(micropplate_x_axis,micropplate_y_axis)
micropplate_z_axis /= np.linalg.norm(micropplate_z_axis)
microplate_position = topleft_midpoint 
microplate_pose = np.eye(4)
microplate_pose[:3,0] = micropplate_x_axis
microplate_pose[:3,1] = micropplate_y_axis
microplate_pose[:3,2] = micropplate_z_axis
microplate_pose[:3,3] = microplate_position

In [ ]:
# grasp in local frame around center and with appropriate orientation.
grasp_pose_in_microplate_frame = SE3Container.from_euler_angles_and_translation([np.pi,0,-np.pi/2],[0.06,-0.04,-0.015]).homogeneous_matrix
grasp_pose_in_microplate_frame[2,3] -= 0.01 # gripper offset for wide grasp 

grasp_pose = microplate_pose @ grasp_pose_in_microplate_frame
import spatialmath.base as sm
grasp_pose = sm.trnorm(grasp_pose)
pre_grasp_pose = np.copy(grasp_pose)
pre_grasp_pose[2,3] += 0.1


Grasp & Lift

In [ ]:
#pose = SE3Container.from_euler_angles_and_translation([np.pi,0,0],topleft_midpoint).homogeneous_matrix
robot.move_linear_to_tcp_pose(pre_grasp_pose,0.05).wait()
gripper.open().wait()
robot.move_linear_to_tcp_pose(grasp_pose,0.05).wait()
gripper.move(0.075).wait() # cannot close grasp because the forces are too high, even with lowest force...

In [ ]:
robot.move_linear_to_tcp_pose(home_pose,0.1).wait()